In [1]:
import fiona
import pandas as pd
import geopandas as gpd
import csv

In [2]:
from collections import OrderedDict
    
layers = fiona.listlayers('ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg')

In [3]:
layers

['grid_1_degree',
 'pollination_table_full',
 'pollination_table',
 'total_ppl_poll',
 'pollination_table_with_negative_ones',
 'region_NCPsummary_positive',
 'region_NCPsummary_negative',
 'production_table',
 'region_PCsummary_positive',
 'region_PCsummary_negative',
 'c_poll_for_violins',
 'd_poll_and_tot_for_violins',
 'relevant_population',
 'c_unrealized',
 'UNxNCP',
 'UNxNCP_ssp1_incrpot',
 'UNxNCP_ssp3_incrpot',
 'UNxNCP_ssp5_incrpot',
 'UNxNCP_ssp1_incrpot_or',
 'UNxNCP_ssp3_incrpot_or',
 'UNxNCP_ssp5_incrpot_or',
 'region_summary_capped',
 'region_summary',
 'region_popimpacted_ssp1',
 'region_popimpacted_ssp3',
 'region_popimpacted_ssp5',
 'region_popveryimpacted_ssp1',
 'region_popveryimpacted_ssp3',
 'region_popveryimpacted_ssp5']

In [4]:
all_datasets = dict()

for layer in layers:
    all_datasets[layer.lower()] = gpd.read_file(
        "ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg", layer=layer
    )

In [5]:
list(all_datasets['relevant_population'])

['fid',
 'GRIDCODE',
 'region',
 'country',
 'prod_total_realized_en_1d_cur',
 'prod_total_realized_fo_1d_cur',
 'prod_total_realized_va_1d_cur',
 'prod_poll_dep_realized_en_1d_cur',
 'prod_poll_dep_realized_fo_1d_cur',
 'prod_poll_dep_realized_va_1d_cur',
 'prod_poll_dep_unrealized_en_1d_cur',
 'prod_poll_dep_unrealized_fo_1d_cur',
 'prod_poll_dep_unrealized_va_1d_cur',
 'nut_req_en_1d_cur',
 'nut_req_fo_1d_cur',
 'nut_req_va_1d_cur',
 'prod_total_realized_en_1d_ssp1',
 'prod_total_realized_fo_1d_ssp1',
 'prod_total_realized_va_1d_ssp1',
 'prod_poll_dep_realized_en_1d_ssp1',
 'prod_poll_dep_realized_fo_1d_ssp1',
 'prod_poll_dep_realized_va_1d_ssp1',
 'prod_poll_dep_unrealized_en_1d_ssp1',
 'prod_poll_dep_unrealized_fo_1d_ssp1',
 'prod_poll_dep_unrealized_va_1d_ssp1',
 'nut_req_en_1d_ssp1',
 'nut_req_fo_1d_ssp1',
 'nut_req_va_1d_ssp1',
 'prod_total_realized_en_1d_ssp3',
 'prod_total_realized_fo_1d_ssp3',
 'prod_total_realized_va_1d_ssp3',
 'prod_poll_dep_realized_en_1d_ssp3',
 'prod_po

### Fields we want to visualize:
NCP_cur,NCP_sspX, UN_cur, UN_sspX, or any of the changes... cNCP_, cUN_
### Preparing the data

en = energy , fo = Folate , va = vitamineA <br>
curr = current<br>
ssp1 = First scenario<br> <br> 
UN = Unmet need (current, ssp1 , ssp3 , ssp5)<br>
NCP = Nature's Contribution to Pollination(current, ssp1 , ssp3 , ssp5)<br>


In [6]:
unxncp = all_datasets['unxncp']

current = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_cur','prod_poll_dep_unrealized_fo_1d_cur','prod_poll_dep_unrealized_va_1d_cur',
                  'nat_cont_poll_en_1d_cur','nat_cont_poll_fo_1d_cur','nat_cont_poll_va_1d_cur','prod_poll_dep_realized_en_1d_cur','prod_poll_dep_realized_fo_1d_cur'
                  ,'prod_poll_dep_realized_va_1d_cur','UN_cur','NCP_cur']]

ssp1 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp1','prod_poll_dep_unrealized_fo_1d_ssp1','prod_poll_dep_unrealized_va_1d_ssp1',
                  'nat_cont_poll_en_1d_ssp1','nat_cont_poll_fo_1d_ssp1','nat_cont_poll_va_1d_ssp1','prod_poll_dep_realized_en_1d_ssp1','prod_poll_dep_realized_fo_1d_ssp1'
                  ,'prod_poll_dep_realized_va_1d_ssp1','UN_ssp1','NCP_ssp1']]

ssp3 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp3','prod_poll_dep_unrealized_fo_1d_ssp3','prod_poll_dep_unrealized_va_1d_ssp3',
                  'nat_cont_poll_en_1d_ssp3','nat_cont_poll_fo_1d_ssp3','nat_cont_poll_va_1d_ssp3','prod_poll_dep_realized_en_1d_ssp3','prod_poll_dep_realized_fo_1d_ssp3'
                  ,'prod_poll_dep_realized_va_1d_ssp3','UN_ssp3','NCP_ssp3']]

ssp5 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp5','prod_poll_dep_unrealized_fo_1d_ssp5','prod_poll_dep_unrealized_va_1d_ssp5',
                  'nat_cont_poll_en_1d_ssp5','nat_cont_poll_fo_1d_ssp5','nat_cont_poll_va_1d_ssp5','prod_poll_dep_realized_en_1d_ssp5','prod_poll_dep_realized_fo_1d_ssp5'
                  ,'prod_poll_dep_realized_va_1d_ssp5','UN_ssp5','NCP_ssp5']]


# Interesting Data 
NCP = unxncp[['GRIDCODE','NCP_cur','NCP_ssp1','NCP_ssp3','NCP_ssp5']]
UN = unxncp[['GRIDCODE','UN_cur','UN_ssp1','UN_ssp3','UN_ssp5']]

# Changes

cNCP = unxncp[['GRIDCODE','cNCP_ssp1','cNCP_ssp3','cNCP_ssp5']]
cUN = unxncp[['GRIDCODE','cUN_ssp1','cUN_ssp3','cUN_ssp5']]



# # Data to csv
# NCP.to_csv('NCP', sep='\t')


In [7]:
UN.head()

,GRIDCODE,UN_cur,UN_ssp1,UN_ssp3,UN_ssp5
0,8846,0.000000,0.002181,0.001248,0.030107
1,9205,479.207931,491.686198,491.623908,649.949682
2,9206,23750.552670,24415.330552,24589.665243,25675.645555
3,9207,630.529173,1248.400221,1264.336585,1646.128832
4,9208,0.000000,0.002589,0.002469,0.002812


In [8]:
all_datasets['pollination_table_full'].head()

,fid,geom,GRIDCODE,country,region,PCTU5,UW,prod_poll_dep_realized_en_1d_ssp5,nat_cont_poll_en_1d_ssp5,prod_poll_dep_unrealized_en_1d_ssp5,...,gpw_v4_e_atot_pop_30s_ssp3,gpw_v4_e_atot_pop_30s_ssp5,isag,c_poll_cont_nut_req_avg_1d_ssp1,c_poll_cont_nut_req_avg_1d_ssp3,c_poll_cont_nut_req_avg_1d_ssp5,d_poll_cont_nut_req_avg_1d_ssp1,d_poll_cont_nut_req_avg_1d_ssp3,d_poll_cont_nut_req_avg_1d_ssp5,geometry
0,8845,GP,8846,Finland,Eurasia,5.7,-999.0,3.492576e+06,0.920932,2.998582e+05,...,123749.796875,173010.359375,1,-0.182394,0.011380,-0.331555,-0.000002,1.438966e-07,-0.000004,None
1,9204,GP,9205,Finland,Eurasia,5.7,-999.0,2.081489e+08,0.821218,4.531472e+07,...,61138.570312,91201.968750,1,-0.121255,0.128436,-0.288955,-0.011524,1.220602e-02,-0.027461,None
2,9205,GP,9206,Finland,Eurasia,5.7,-999.0,1.755138e+09,0.523101,1.600121e+09,...,32481.710938,49574.121094,1,-0.009892,0.008775,-0.021472,-0.003538,3.138304e-03,-0.007679,None
3,9206,GP,9207,Finland,Eurasia,5.7,-999.0,4.447526e+08,0.701118,1.895948e+08,...,22747.699219,34888.214844,1,-0.040155,0.001025,-0.269395,-0.013775,3.516674e-04,-0.092417,None
4,9207,GP,9208,Finland,Eurasia,5.7,-999.0,1.596864e+06,0.982765,2.800423e+04,...,39685.875000,58809.265625,1,-0.108285,0.139090,-0.232960,-0.000002,2.028642e-06,-0.000003,None


**Description of the data** <br>

X1 = prod_poll_dep_unrealized (The unrealized production that is pollination dependant)<br>
X2 = prod_poll_dep_realized ( The realized production that is pollination dependant) <br>
Y = prod_poll_indep (production that is pollination independant)<br>
Z = nut_req ( nutrition required for the area)<br>

**Data Available for these timeframes:** 1850 - 1900 - 1910 - 1945 - 1980 - current - ssp1 - ssp3 - ssp5

In [9]:
pollination_table = all_datasets['pollination_table_full']

#  1850
# Z is not available for this year
data_1850 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1850','prod_poll_dep_unrealized_va_1d_1850','prod_poll_dep_unrealized_fo_1d_1850',
                               'prod_poll_dep_realized_en_1d_1850','prod_poll_dep_realized_va_1d_1850','prod_poll_dep_realized_fo_1d_1850',
                               'prod_total_realized_en_1d_1850','prod_total_realized_va_1d_1850','prod_total_realized_fo_1d_1850']]
data_1850.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1900
# Z is not available for this year
data_1900 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1900','prod_poll_dep_unrealized_va_1d_1900','prod_poll_dep_unrealized_fo_1d_1900',
                               'prod_poll_dep_realized_en_1d_1900','prod_poll_dep_realized_va_1d_1900','prod_poll_dep_realized_fo_1d_1900',
                               'prod_total_realized_en_1d_1900','prod_total_realized_va_1d_1900','prod_total_realized_fo_1d_1900']]
data_1900.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1910
# Z is not available for this year
data_1910 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1910','prod_poll_dep_unrealized_va_1d_1910','prod_poll_dep_unrealized_fo_1d_1910',
                               'prod_poll_dep_realized_en_1d_1910','prod_poll_dep_realized_va_1d_1910','prod_poll_dep_realized_fo_1d_1910',
                               'prod_total_realized_en_1d_1910','prod_total_realized_va_1d_1910','prod_total_realized_fo_1d_1910']]
data_1910.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1945
# Z is not available for this year
data_1945 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1945','prod_poll_dep_unrealized_va_1d_1945','prod_poll_dep_unrealized_fo_1d_1945',
                               'prod_poll_dep_realized_en_1d_1945','prod_poll_dep_realized_va_1d_1945','prod_poll_dep_realized_fo_1d_1945',
                               'prod_total_realized_en_1d_1945','prod_total_realized_va_1d_1945','prod_total_realized_fo_1d_1945']]
data_1945.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1980
# Z is not available for this year
data_1980 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1980','prod_poll_dep_unrealized_va_1d_1980','prod_poll_dep_unrealized_fo_1d_1980',
                               'prod_poll_dep_realized_en_1d_1980','prod_poll_dep_realized_va_1d_1980','prod_poll_dep_realized_fo_1d_1980',
                               'prod_total_realized_en_1d_1980','prod_total_realized_va_1d_1980','prod_total_realized_fo_1d_1980']]
data_1980.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']
data_1980

#  cur
data_cur = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_cur','prod_poll_dep_unrealized_va_1d_cur','prod_poll_dep_unrealized_fo_1d_cur',
                               'prod_poll_dep_realized_en_1d_cur','prod_poll_dep_realized_va_1d_cur','prod_poll_dep_realized_fo_1d_cur',
                               'prod_total_realized_en_1d_cur','prod_total_realized_va_1d_cur','prod_total_realized_fo_1d_cur',
                               'nut_req_en_1d_cur','nut_req_va_1d_cur','nut_req_fo_1d_cur']]

data_cur.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp1
data_ssp1 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp1','prod_poll_dep_unrealized_va_1d_ssp1','prod_poll_dep_unrealized_fo_1d_ssp1',
                               'prod_poll_dep_realized_en_1d_ssp1','prod_poll_dep_realized_va_1d_ssp1','prod_poll_dep_realized_fo_1d_ssp1',
                               'prod_total_realized_en_1d_ssp1','prod_total_realized_va_1d_ssp1','prod_total_realized_fo_1d_ssp1',
                               'nut_req_en_1d_ssp1','nut_req_va_1d_ssp1','nut_req_fo_1d_ssp1']]

data_ssp1.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp3
data_ssp3 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp3','prod_poll_dep_unrealized_va_1d_ssp3','prod_poll_dep_unrealized_fo_1d_ssp3',
                               'prod_poll_dep_realized_en_1d_ssp3','prod_poll_dep_realized_va_1d_ssp3','prod_poll_dep_realized_fo_1d_ssp3',
                               'prod_total_realized_en_1d_ssp3','prod_total_realized_va_1d_ssp3','prod_total_realized_fo_1d_ssp3',
                               'nut_req_en_1d_ssp3','nut_req_va_1d_ssp3','nut_req_fo_1d_ssp3']]

data_ssp3.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp5
data_ssp5 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp5','prod_poll_dep_unrealized_va_1d_ssp5','prod_poll_dep_unrealized_fo_1d_ssp5',
                               'prod_poll_dep_realized_en_1d_ssp5','prod_poll_dep_realized_va_1d_ssp5','prod_poll_dep_realized_fo_1d_ssp5',
                               'prod_total_realized_en_1d_ssp5','prod_total_realized_va_1d_ssp5','prod_total_realized_fo_1d_ssp5',
                               'nut_req_en_1d_ssp5','nut_req_va_1d_ssp5','nut_req_fo_1d_ssp5']]

data_ssp5.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']





In [10]:
def add_iso_columns (dataFrame,country_iso,name):
    dataFrame = pd.merge(dataFrame, country_iso,left_on=name,right_on='English short name lower case', how='outer').head(dataFrame.shape[0])
    dataFrame= dataFrame.drop(columns=['English short name lower case', 'Alpha-2 code','Numeric code','ISO 3166-2','Unnamed: 0'])
    dataFrame = dataFrame.rename(index=str, columns={"Alpha-3 code": "iso3"})

    cols = dataFrame.columns.tolist()
    cols = [cols[0]] + [cols[-1]] + cols[1:-1]
    dataFrame = dataFrame[cols]
    return dataFrame

def add_iso (dataFrame,country_iso,name):
    dataFrame = dataFrame.groupby([name]).sum()
    dataFrame.reset_index(level=0, inplace=True)
    dataFrame= add_iso_columns(dataFrame,country_iso,name)
    return dataFrame
    
def create_csv(dataFrame,country_iso, name):
    dataFrame = add_iso(dataFrame,country_iso, name)
    dataFrame.to_csv(name +'.csv', sep=',')
    
def create_all_csv(data_folder):
    create_csv (data_1850,country_iso, data_folder+'1850')
    create_csv (data_1900,country_iso, data_folder+'1900')
    create_csv (data_1910,country_iso, data_folder+'1910')
    create_csv (data_1945,country_iso, data_folder+'1945')
    create_csv (data_1980,country_iso, data_folder+'1980')
    create_csv (data_cur,country_iso, data_folder+'current')
    create_csv (data_ssp1,country_iso, data_folder+'ssp1')
    create_csv (data_ssp3,country_iso, data_folder+'ssp3')
    create_csv (data_ssp5,country_iso, data_folder+'ssp5')
    


In [11]:
country_iso = pd.read_csv("./data/country_iso.csv")
data_folder = 'data_iso/'
# create_all_csv(data_folder)

In [12]:
# X2+Y folate for all the periods
mock_data = pd.DataFrame(index=data_1850.index)
mock_data = pd.concat([data_1850.name,data_1850['X2+Y_fo'],
                      data_1900['X2+Y_fo'],data_1910['X2+Y_fo'],
                      data_1945['X2+Y_fo'],data_1980['X2+Y_fo'],
                      data_cur['X2+Y_fo'],data_ssp1['X2+Y_fo'],
                      data_ssp3['X2+Y_fo'],data_ssp5['X2+Y_fo'],], axis=1)
mock_data.columns = ['name', '1850','1900', '1910','1945', '1980','cur', 'ssp1','ssp3', 'ssp5']
mock_data = add_iso (mock_data,country_iso, data_folder+'mock_data')
mock_data.to_csv('mock_data' +'.csv', sep=',')

world_data = pd.DataFrame(mock_data.sum()).transpose().drop(columns=['name','iso3'])
world_data.to_csv('world_data' +'.csv', sep=',', float_format='{:.2E}'.format)
mock_data.head()

KeyError: 'data_iso/mock_data'

In [ ]:
world_data

Questions:

1. What does the GRIDCODE actually mean? and if they are the regions how can we relate them to latitude and longitude?
2. SSP1, SSP3, SSP5 are they the future scenario description? but how are these scenarios described?
3. Difference between semi natural and natural habitats?
4. 10s – globio, 30s – gpw, monfreda 2008 – 5m, ssp – 1/8d) clearing scales?
5. What does `c_` prefix in front of columns mean?

Observations:

- GRIDCODE is unique.

In [ ]:
latlong = pd.read_csv("./data/latlong_pollination.csv")
latlong.columns = ['fid', 'long','lat']
latlong.tail()

In [ ]:
pollination_table = all_datasets['pollination_table_full'].merge(latlong, left_on='fid', right_on='fid', how='inner')

In [ ]:
pollination_table[['country','prod_poll_dep_realized_en_1d_ssp5','nat_cont_poll_en_1d_ssp5','prod_poll_dep_unrealized_en_1d_ssp5','prod_total_realized_en_1d_ssp5','nut_req_en_1d_ssp5','poll_cont_nut_req_en_1d_ssp5']].head()


In [ ]:
unxncp = all_datasets['unxncp'].merge(latlong, left_on='fid', right_on='fid', how='inner')

Nature's contribution to pollination is the percentage of prod_poll_dep_realized/ the total production

In [36]:
unxncp[['prod_poll_dep_unrealized_en_1d_cur', 'nat_cont_poll_en_1d_cur', 'prod_poll_dep_realized_en_1d_cur','UN_cur','NCP_cur']].head()

,prod_poll_dep_unrealized_en_1d_cur,nat_cont_poll_en_1d_cur,prod_poll_dep_realized_en_1d_cur,UN_cur,NCP_cur
0,0.000000e+00,1.000000,3.792434e+06,0.000000,0.333333
1,3.076646e+07,0.878616,2.226972e+08,479.207931,0.859045
2,1.399546e+09,0.587343,1.992002e+09,23750.552670,0.522512
3,4.935109e+07,0.922920,5.909089e+08,630.529173,0.896613
4,0.000000e+00,1.000000,1.624868e+06,0.000000,1.000000


# PERCENTAGES

### For points on the map

In [65]:
resource = 'en'
resolution = '1d'
periods = ['1850','1900','1910','1945','1980','cur','ssp1','ssp3','ssp5']

In [84]:
columns_totake = []
for period in periods :
    columns_totake.append('nat_cont_poll_'+resource+'_'+resolution+'_'+period)

columns_totransform = columns_totake.copy()
columns_totake.extend(('long', 'lat'))

natures_contribution_pixels = pollination_table[['fid','country']]
natures_contribution_pixels = pd.concat([natures_contribution_pixels, pollination_table[columns_totake].copy()], axis=1)
for col in columns_totransform:
    natures_contribution_pixels[col] = (natures_contribution_pixels[col]*10000).round()/100

column_names = ['fid','country']
for col in periods:
    column_names.append(col)

column_names.append('long')
column_names.append('lat')
for i in range(7,10):
    column_names[i] = column_names[i].upper()

column_names[6] = '2015'
natures_contribution_pixels.columns = column_names
natures_contribution_pixels.head()


,fid,country,1850,1900,1910,1945,2015,CUR,SSP1,SSP3,ssp5,long,lat
0,8845,Finland,100.00,100.00,100.00,100.00,100.00,100.00,99.43,99.67,92.09,65.5,25.5
1,9204,Finland,87.16,80.55,74.59,71.56,85.38,87.86,87.60,87.60,82.12,64.5,24.5
2,9205,Finland,58.77,55.52,53.28,51.56,56.77,58.73,55.75,55.96,52.31,64.5,25.5
3,9206,Finland,93.28,90.92,90.91,90.81,90.08,92.29,79.11,79.30,70.11,64.5,26.5
4,9207,Finland,100.00,100.00,100.00,100.00,100.00,100.00,98.41,98.49,98.28,64.5,27.5


In [85]:
natures_contribution_pixels.head(5)

,fid,country,1850,1900,1910,1945,2015,CUR,SSP1,SSP3,ssp5,long,lat
0,8845,Finland,100.00,100.00,100.00,100.00,100.00,100.00,99.43,99.67,92.09,65.5,25.5
1,9204,Finland,87.16,80.55,74.59,71.56,85.38,87.86,87.60,87.60,82.12,64.5,24.5
2,9205,Finland,58.77,55.52,53.28,51.56,56.77,58.73,55.75,55.96,52.31,64.5,25.5
3,9206,Finland,93.28,90.92,90.91,90.81,90.08,92.29,79.11,79.30,70.11,64.5,26.5
4,9207,Finland,100.00,100.00,100.00,100.00,100.00,100.00,98.41,98.49,98.28,64.5,27.5


In [86]:
natures_contribution_pixels = add_iso_columns (natures_contribution_pixels,country_iso,'country').drop(columns=['country'])
natures_contribution_pixels['iso3'] = natures_contribution_pixels['iso3'] + natures_contribution_pixels['fid'].astype(str)
natures_contribution_pixels.to_csv('./data/pixel_energy.csv', sep=',')
natures_contribution_pixels.head()

,fid,iso3,1850,1900,1910,1945,2015,CUR,SSP1,SSP3,ssp5,long,lat
0,8845.0,FIN8845.0,100.00,100.00,100.00,100.00,100.00,100.00,99.43,99.67,92.09,65.5,25.5
1,9204.0,FIN9204.0,87.16,80.55,74.59,71.56,85.38,87.86,87.60,87.60,82.12,64.5,24.5
2,9205.0,FIN9205.0,58.77,55.52,53.28,51.56,56.77,58.73,55.75,55.96,52.31,64.5,25.5
3,9206.0,FIN9206.0,93.28,90.92,90.91,90.81,90.08,92.29,79.11,79.30,70.11,64.5,26.5
4,9207.0,FIN9207.0,100.00,100.00,100.00,100.00,100.00,100.00,98.41,98.49,98.28,64.5,27.5


### Find min max latitudes

In [41]:
mins = natures_contribution_pixels.groupby('country').idxmin().add_suffix('_min')

maxs = natures_contribution_pixels.groupby('country').idxmax().add_suffix('_max')
countriesdf = natures_contribution_pixels.groupby('country').count().reset_index()
mins.head()

,1850_min,1900_min,1910_min,1945_min,1980_min,2015_min,SSP1_min,SSP3_min,SSP5_min,lat_min,long_min
country,,,,,,,,,,,
Afghanistan,3221,3221,3377,3377,3534,3534,3534,3534,3534,3949,4397
Albania,2707,2707,2555,2707,2707,2707,2707,2707,2707,2554,2861
Algeria,3925,3648,3648,3648,3648,3648,3481,3648,3648,3642,3925
Angola,7294,7294,7294,7389,7489,7776,7776,7489,7996,7293,8432
Argentina,8581,8581,8581,8640,8640,8640,8640,8640,8640,9501,9508


In [42]:
natures_contribution_pixels.iloc[2707,:]

country    Albania
1850          8.27
1900          5.77
1910          6.26
1945          6.61
1980          7.64
2015           9.3
SSP1         11.92
SSP3         11.66
SSP5          9.49
long          41.5
lat           19.5
Name: 2707, dtype: object

In [43]:
fids = pollination_table[['fid','lat','long']]
natures_contribution_pixels_min_max = pd.DataFrame(countriesdf['country'])
natures_contribution_pixels_min_max

periodnames = ['1850', '1900', '1910', '1945', '1980', '2015', 'SSP1', 'SSP3', 'SSP5']

for period in periodnames:
    natures_contribution_pixels_min_max[period+'_lat_min'] = pd.DataFrame(fids.iloc[mins[period+'_min'],:].lat).reset_index().drop(columns=['index'])
    natures_contribution_pixels_min_max[period+'_long_min'] = pd.DataFrame(fids.iloc[mins[period+'_min'],:].long).reset_index().drop(columns=['index'])
    natures_contribution_pixels_min_max[period+'_lat_max'] = pd.DataFrame(fids.iloc[maxs[period+'_max'],:].lat).reset_index().drop(columns=['index'])
    natures_contribution_pixels_min_max[period+'_long_max'] = pd.DataFrame(fids.iloc[maxs[period+'_max'],:].long).reset_index().drop(columns=['index'])

natures_contribution_pixels_min_max.head()


,country,1850_lat_min,1850_long_min,1850_lat_max,1850_long_max,1900_lat_min,1900_long_min,1900_lat_max,1900_long_max,1910_lat_min,...,SSP1_lat_max,SSP1_long_max,SSP3_lat_min,SSP3_long_min,SSP3_lat_max,SSP3_long_max,SSP5_lat_min,SSP5_long_min,SSP5_lat_max,SSP5_long_max
0,Afghanistan,71.5,38.5,70.5,37.5,71.5,38.5,70.5,37.5,73.5,...,73.5,37.5,63.5,36.5,74.5,37.5,63.5,36.5,74.5,37.5
1,Albania,19.5,41.5,19.5,42.5,19.5,41.5,19.5,42.5,20.5,...,19.5,42.5,19.5,41.5,19.5,42.5,19.5,41.5,19.5,42.5
2,Algeria,3.5,33.5,2.5,34.5,3.5,35.5,5.5,33.5,3.5,...,-2.5,35.5,3.5,35.5,7.5,33.5,3.5,35.5,5.5,33.5
3,Angola,13.5,-4.5,12.5,-4.5,13.5,-4.5,12.5,-4.5,13.5,...,15.5,-6.5,15.5,-6.5,22.5,-9.5,16.5,-12.5,20.5,-12.5
4,Argentina,-65.5,-21.5,-64.5,-22.5,-65.5,-21.5,-63.5,-21.5,-65.5,...,-62.5,-21.5,-66.5,-22.5,-62.5,-22.5,-66.5,-22.5,-62.5,-22.5


In [44]:
natures_contribution_pixels_min_max.to_csv('min_max_coordinates_by_country.csv',sep=',')

### For countries on the map

In [45]:
natures_contribution_countries = pollination_table[['country']].copy()
natures_contribution_countries.columns= ['name']

for period in periods:
        natures_contribution_countries['realized'+period] = pollination_table['prod_poll_dep_realized_en_1d_'+period]
        natures_contribution_countries['unrealized'+period] = pollination_table['prod_poll_dep_unrealized_en_1d_'+period]

#     natures_contribution_countries[period] = (pollination_table['prod_poll_dep_realized_en_1d_'+period]/(pollination_table['prod_poll_dep_realized_en_1d_'+period] +pollination_table['prod_poll_dep_unrealized_en_1d_'+period])*10000).round()/100
natures_contribution_countries.sample(5)


,name,realized1850,unrealized1850,realized1900,unrealized1900,realized1910,unrealized1910,realized1945,unrealized1945,realized1980,unrealized1980,realizedcur,unrealizedcur,realizedssp1,unrealizedssp1,realizedssp3,unrealizedssp3,realizedssp5,unrealizedssp5
213,Norway,954775.250,2605172.25,5.819071e+05,3.330074e+06,5.428510e+05,3.369130e+06,4.578338e+05,3.454147e+06,1.294126e+06,2.617855e+06,3.255998e+06,3.004038e+05,3.420744e+06,1.356569e+05,3.312410e+06,2.439917e+05,3.292798e+06,1.891263e+05
1833,United States,-1.000,-1.00,3.207459e+07,0.000000e+00,9.663315e+07,0.000000e+00,9.344620e+07,3.186956e+06,2.306608e+08,3.437882e+06,3.291534e+07,0.000000e+00,3.067490e+07,0.000000e+00,9.060310e+07,4.498942e+06,9.464335e+07,3.387398e+04
3775,United States,-1.000,-1.00,5.671900e+09,7.339396e+08,1.087605e+10,3.429694e+09,4.519857e+09,2.698829e+10,5.203283e+09,2.630487e+10,3.457472e+09,1.084828e+10,3.093576e+09,2.839039e+09,2.007499e+09,1.198015e+10,2.675981e+09,1.100880e+10
2608,China,744698.375,1326027.50,2.013636e+06,1.664161e+06,3.391671e+05,7.064836e+06,9.967435e+05,1.398821e+07,7.066785e-11,1.498495e+07,1.419656e-10,3.132143e+07,7.068129e-11,1.498495e+07,3.513347e-11,7.404004e+06,1.629236e-10,3.132143e+07
2016,Canada,-1.000,-1.00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00


In [46]:
natures_contribution_countries =add_iso (natures_contribution_countries,country_iso, 'name')

In [47]:
final_natures_cont_countries = natures_contribution_countries[['name','iso3']].copy()
for period in periods:
    final_natures_cont_countries[period] = ((natures_contribution_countries['realized'+period]/(natures_contribution_countries['realized'+period] +natures_contribution_countries['unrealized'+period]))*10000).fillna(50).astype(int)/100.


column_names = final_natures_cont_countries.columns.tolist()

for i in range(8,11):
    column_names[i] = column_names[i].upper()
    
column_names[7] = '2015'
final_natures_cont_countries.columns = column_names

columns_tochange = ['1850','1900','1910','1945','1980','2015','SSP1','SSP3','SSP5']
for col in columns_tochange:
    final_natures_cont_countries[col][final_natures_cont_countries[col] < 0] = 0
final_natures_cont_countries.head()



/anaconda3/envs/dataviz/lib/python3.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,iso3,1850,1900,1910,1945,1980,2015,SSP1,SSP3,SSP5
0,Afghanistan,AFG,7.67,7.48,8.16,7.70,4.00,3.79,5.10,3.17,3.86
1,Albania,ALB,46.71,13.35,12.45,16.28,29.17,48.45,52.56,52.25,45.29
2,Algeria,DZA,20.62,8.44,6.72,3.64,1.92,1.31,0.67,0.75,1.14
3,Angola,AGO,57.73,72.24,75.91,82.82,86.34,80.87,76.41,26.52,52.63
4,Argentina,ARG,45.93,30.83,22.39,7.24,5.14,3.47,4.39,2.81,2.94


In [48]:
final_natures_cont_countries.to_csv('country_energy.csv', sep=',')

In [50]:
final_natures_cont_countries.tail()

,name,iso3,1850,1900,1910,1945,1980,2015,SSP1,SSP3,SSP5
157,Venezuela,VEN,60.71,53.21,52.71,26.24,15.17,8.50,14.28,5.50,6.97
158,Viet Nam,VNM,45.78,41.49,40.46,37.81,30.84,19.04,21.31,19.22,25.24
159,Yemen,YEM,4.95,6.68,6.57,7.51,7.68,6.09,2.24,0.45,3.65
160,Zambia,ZMB,96.01,96.31,96.90,94.44,87.93,82.68,81.89,40.39,52.65
161,Zimbabwe,ZWE,0.03,0.06,0.09,0.47,1.06,1.46,1.39,2.23,2.02


# Population

In [5]:
population = pd.read_csv('./data/population.csv')

In [6]:
population.head()

,Country Name,Country Code,Year,Value
0,Arab World,ARB,1960,92490932.0
1,Arab World,ARB,1961,95044497.0
2,Arab World,ARB,1962,97682294.0
3,Arab World,ARB,1963,100411076.0
4,Arab World,ARB,1964,103239902.0


In [7]:
pop1980 = population[population['Year'] == 1980].reset_index().drop(columns = ['index','Year'])
pop1980 = pop1980.iloc[45:]

In [8]:
pop1980 = pop1980.rename(columns={'Country Name': 'country', 'Country Code': 'code', 'Value': 'pop'})

In [11]:
pop = all_datasets['relevant_population'][['country','relevant_pop_cur','relevant_pop_ssp1','relevant_pop_ssp3','relevant_pop_ssp5']]
country_iso = pd.read_csv("./data/country_iso.csv")
pop = add_iso (pop,country_iso,'country')
pop.columns = ['country','iso3','2015','SSP1','SSP3','SSP5']
pop.loc[161] = pop.sum()
pop.loc[161,0:2]= ['World','WLD'] 
pop = pd.merge(pop, pop1980,left_on='iso3',right_on='code', how='inner').drop(columns = ['country_y','code'])
pop.rename(columns={'country_x': 'country', 'iso3': 'code','pop':'1980'}, inplace=True)

pop = pop[['country', 'code', '1980', '2015', 'SSP1','SSP3','SSP5']]


In [12]:
pop.head()

,country,code,1980,2015,SSP1,SSP3,SSP5
0,Afghanistan,AFG,13248370.0,42481012.0,70357193.0,102034359.0,66895201.0
1,Albania,ALB,2671997.0,3826103.0,3854961.0,4179058.0,3823691.0
2,Algeria,DZA,19337715.0,22887590.0,30695299.0,40399040.0,32338297.0
3,Angola,AGO,8929900.0,26022638.0,51577488.0,65569416.0,52003575.0
4,Argentina,ARG,28105888.0,29702439.0,33547670.0,41523818.0,32027941.0


pop.to_csv('./data/population' +'.csv', sep=',')

In [32]:
import os
for f in os.listdir('./data/flags'):
    path = os.path.join('./data/flags', f)
    if ( f[0:2] in country_iso.index):
        os.rename(path, f.replace(f[0:2], country_iso.loc[f[0:2],'Alpha-3 code'])) 

In [28]:
country_iso['iso2'] = country_iso['Alpha-2 code'].str.lower()
country_iso =country_iso.set_index('iso2')

'AFG'

In [30]:
os.rename(f, f.replace(f[0:2], country_iso.loc[f[0:2],'Alpha-3 code'])) 

FileNotFoundError: [Errno 2] No such file or directory: 'tg.png' -> 'TGO.png'

# For graph:

In [20]:
res = 'en'
country = pd.read_csv("./data/country_"+ res+".csv",index_col=False).drop(columns = ['Unnamed: 0']) 
unmet_need = pd.read_csv("./data/unmet_need_"+ res+".csv",index_col=None).drop(columns = ['Unnamed: 0']) 


In [21]:
unmet_need.loc[162] = unmet_need.mean()
unmet_need.country.iloc[-1] = 'World'
unmet_need.iso3.iloc[-1] = 'WLD'

In [22]:
unmet_need.to_csv('./data/unmet_need'+res +'.csv', sep=',')

In [195]:
unmet_need = unmet_need.T.reset_index().iloc[1:]
unmet_need.columns = unmet_need.iloc[0]
unmet_need = unmet_need.iloc[1:]
unmet_need = unmet_need.rename(columns={'iso3': 'date'})
unmet_need.iloc[5,0] = '2015'
unmet_need.insert(loc=0, column='name', value='Unmet need')

In [196]:
unmet_need

1,name,date,AFG,ALB,DZA,AGO,ARG,ARM,ABW,AUS,...,TZA,USA,URY,UZB,VEN,VNM,YEM,ZMB,ZWE,WLD
2,Unmet need,1850,48.59,68.02,57.9,47.94,30.64,85.58,50,49.29,...,63.64,39.62,41.46,89.65,43.55,31.12,52.03,46.28,85.26,49.782
3,Unmet need,1900,50.08,82.93,76.34,48.62,40.05,90.92,50,58.77,...,65.18,52.14,78.56,90.69,38.44,39.69,58.26,44.82,86.97,53.3798
4,Unmet need,1910,50.02,84.13,78.14,47.92,46.43,91.57,50,65.33,...,63.91,55.66,86.41,90.63,36.38,44.36,58.83,44.34,88.68,54.441
5,Unmet need,1945,48.31,82.03,81.15,49.11,58.5,91.49,50,75.28,...,73.63,68.76,86.6,99.59,54.01,49.61,61.39,43.53,80.96,57.0441
6,Unmet need,1980,54.04,75.95,90.91,50.14,67.28,91.02,50,84.37,...,80.87,68.41,86.79,99.61,64.27,56.22,58.69,45.45,82.41,61.0209
7,Unmet need,2015,54.95,66.72,91.27,47.93,71.94,92.4,50,85.67,...,83.64,66.11,90.71,99.61,70.24,72.35,73.06,49.79,83.3,62.5442
8,Unmet need,ssp1,53.82,63.66,98.94,36.11,68.14,91.59,50,86.25,...,80.4,62.77,90.85,99.61,64.97,70.01,85.5,44.75,83.91,60.3802
9,Unmet need,ssp3,63.3,64,98.45,93.48,73.87,92.71,50,86.82,...,89.84,70.09,91.42,99.61,72.65,72.37,88.67,78.65,87.47,67.3959
10,Unmet need,ssp5,59,68.37,92.19,55.24,73.03,92.52,50,85.13,...,86.08,67.61,91.05,99.62,71.27,65.41,76.72,61.5,86.01,65.3698


In [197]:
country = country.T.reset_index().iloc[1:]
country.columns = country.iloc[0]
country = country.iloc[1:]
country = country.rename(columns={'iso3': 'date'})
country.iloc[5,0] = '2015'
country.insert(loc=0, column='name', value='Pollination Contribution')

In [198]:
country

1,name,date,AFG,ALB,DZA,AGO,ARG,ARM,ABW,AUS,...,TZA,USA,URY,UZB,VEN,VNM,YEM,ZMB,ZWE,WLD
2,Pollination Contribution,1850,9.47,41.63,19.69,31.65,43.68,12.99,50,26.19,...,13.62,37.51,99.59,0.74,66.35,45.96,5.52,96.93,0.04,43.2747
3,Pollination Contribution,1900,9.22,12.24,7.93,42.27,30.19,8.82,50,22.14,...,13.05,15.35,3.96,0.28,57.53,41.36,7.46,95.47,0.08,38.0683
4,Pollination Contribution,1910,9.85,11.48,6.3,45.61,21.84,8.45,50,18.86,...,12.99,13.27,1.64,0.34,56.03,40.43,7.35,95.19,0.11,36.7769
5,Pollination Contribution,1945,8.29,14.99,3.34,62.68,6.58,7.74,50,1.05,...,11.55,7.71,1.27,0.46,28.43,37.67,8.4,86.99,0.54,32.4202
6,Pollination Contribution,1980,3.76,26.83,1.9,77.86,4.49,7.39,0.5,2.76,...,9.06,7.49,1.53,0.43,16.78,30.56,8.57,72.51,1.15,27.213
7,Pollination Contribution,2015,3.51,43.67,1.25,74.68,2.98,4.93,0.5,2.76,...,8.87,8.49,1.6,0.43,9.46,19.23,6.8,67.11,1.55,25.1781
8,Pollination Contribution,SSP1,5.03,47.59,0.7,67.02,4.06,5.72,0.5,2.76,...,12.04,9.56,1.5,0.43,15.72,21.62,2.5,68.37,1.49,28.3099
9,Pollination Contribution,SSP3,2.85,47.3,0.76,17.91,2.34,3.77,0.5,2.84,...,4.9,6.87,1.47,0.43,6.3,19.49,0.5,16.04,2.21,22.7452
10,Pollination Contribution,SSP5,3.5,40.87,1.06,45.73,2.44,5.64,0.5,2.75,...,8.98,7.83,1.5,0.42,7.8,25.77,4.06,30.94,2.04,23.2894


In [199]:
data = pd.concat([unmet_need,country], axis=0)
data.to_csv('./data/plot'+res +'.csv', sep=',')